# Project 1: Build an LLM Playground

Welcome to your first project! In this project, you'll build a simple large language model (LLM) playground, an interactive environment where you can experiment with LLMs and understand how they work under the hood.

The goal here is to understand the foundations and mechanics behind LLMs rather than relying on higher-level abstractions or frameworks. You'll see what happens “under the hood”, how an LLM receives a text, processes it, and generate a response. In later projects, you'll use frameworks like Ollama and LangChain that simplify many of these steps. But before that, this project will help you build a solid mental model of how LLMs actually work.

We'll use Google Colab, a free browser-based platform that lets you run Python code and machine learning models without installing anything locally. Click the button below to open this notebook in Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bytebyteai/ai-eng-projects-2/blob/main/project_1/lm_playground.ipynb)

If you prefer to run the project locally, you can use the provided `env.yaml` file to create a compatible environment using conda. To do so, open a terminal in the same directory as this notebook and run:

```bash
# Create and activate the conda environment
conda env create -f env.yaml && conda activate llm_playground

# Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=llm_playground --display-name "llm_playground"
```


---
## Learning Objectives  
- Understand tokenization and how raw text is converted into a sequence of discrete tokens
- Inspect GPT-2 and the Transformer architecture
- Learn how to load pretrained LLMs using Hugging Face
- Explore decoding strategies to generate text from LLMs
- Compare completion models with instruction-tuned models


Let's get started!

In [ ]:
# Confirm required libraries are installed and working.
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__)
print("✅ Environment check complete. You're good to go!")

# 1 - Tokenization

A neural network cannot process raw text directly. It needs numbers.
Tokenization is the process of converting text into numerical IDs that models can understand. In this section, you will learn how tokenization works in practice and why it is an essential step in every language model pipeline.

Tokenization methods generally fall into three main categories:
1. Word-level
2. Character-level
3. Subword-level

### 1.1 - Word-level tokenization
This method splits text by whitespace and treats each word as a single token. In the next cell, you will implement a basic word-level tokenizer by building a vocabulary that maps words to IDs and writing `encode` and `decode` functions.

In [1]:
# Creating a tiny corpus. In practice, a corpus is generally the entire internet-scale dataset used for training.
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

# Step 1: Build vocabulary (all unique words in the corpus) and mappings
vocab = []
word2id = {}
id2word = {}

# Collect tokens (lowercased)
tokens = []
for sent in corpus:
    for w in sent.lower().split():
        tokens.append(w)

# Build sorted unique vocabulary
vocab = sorted(set(tokens))

# Create lookup mappings
word2id = {w: i for i, w in enumerate(vocab)}
id2word = {i: w for w, i in word2id.items()}

print(f"Vocabulary size: {len(vocab)} words")
print("First 15 vocab entries:", vocab[:15])


Vocabulary size: 19 words
First 15 vocab entries: ['brown', 'converts', 'dog', 'fox', 'jumps', 'language', 'large', 'lazy', 'models', 'next', 'numbers', 'over', 'predict', 'quick', 'text']


In [4]:
# Step 2: Define encode and decode functions
def encode(text):
    # converts text to token IDs
    return [word2id[w] for w in text.lower().split()]

    pass


def decode(ids):
    # converts token IDs back to text
       return " ".join(id2word[i] for i in ids)
       pass

In [5]:
# Step 3: Test your tokenizer with random sentences.
# Try a sentence with unseen words and see what happens (and how to fix it)
# Example sentences to test
examples = [
    "The quick brown fox",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

for s in examples:
    ids = encode(s)
    back = decode(ids)
    print(f"Input:   {s}")
    print(f"IDs:     {ids}")
    print(f"Decoded: {back}")
    print("-" * 40)

Input:   The quick brown fox
IDs:     [15, 13, 0, 3]
Decoded: the quick brown fox
----------------------------------------
Input:   Tokenization converts text to numbers
IDs:     [18, 1, 14, 16, 10]
Decoded: tokenization converts text to numbers
----------------------------------------
Input:   Large language models predict the next token
IDs:     [6, 5, 8, 12, 15, 9, 17]
Decoded: large language models predict the next token
----------------------------------------


While word-level tokenization is simple and easy to understand, it has two key limitations that make it impractical for large-scale models:
1.  large vocabulary size: every new word or variation (for example, run, runs, running) increases the total vocabulary, leading to higher memory and training costs.
2. Out-of-vocabulary (OOV) problem: the model cannot handle unseen or rare words that were not part of the training vocabulary, so they must be replaced with a generic [UNK] token.

The next section introduces character-level tokenization, where text is represented as individual characters instead of words.

### 1.2 - Character-level tokenization

In this approach, every single character (including spaces, punctuation, and even emojis) is assigned its own ID.

In the next section, we will rebuild a tokenizer using the same corpus as before, but this time with a character-level approach.
For simplicity, assume we are only using lowercase and uppercase English letters (a-z, A-Z).

In [6]:
import string

# Step 1: Create a vocabulary that includes all uppercase and lowercase letters.
vocab = []
char2id = {}
id2char = {}
# Add two special tokens first
vocab = ["<pad>", "<unk>"]

# Add all uppercase A–Z
vocab += list(string.ascii_uppercase)

# Add all lowercase a–z
vocab += list(string.ascii_lowercase)

# Build lookup tables
char2id = {ch: i for i, ch in enumerate(vocab)}
id2char = {i: ch for ch, i in char2id.items()}

print(f"Vocabulary size: {len(vocab)} (52 letters + 2 specials)")


Vocabulary size: 54 (52 letters + 2 specials)


In [7]:
# Step 2: Implement encode() and decode() functions to convert between text and IDs.
def encode(text):
    # convert text to list of IDs
    return [char2id.get(ch, char2id["<unk>"]) for ch in text]

    pass


def decode(ids):
    # Convert list of IDs to text
    return "".join(id2char[i] for i in ids if id2char[i] != "<pad>")
    pass

In [8]:
# Step 3: Test your tokenizer on a short sample word.
# --- Test the character tokenizer ---

test_strings = [
    "HelloWorld",
    "ABCxyz",
    "ChatGPT",
    "Hello, World!"  # note: comma and space will become <unk>
]

for s in test_strings:
    ids = encode(s)
    decoded = decode(ids)
    print(f"Original: {s}")
    print(f"Encoded : {ids}")
    print(f"Decoded : {decoded}")
    print("-" * 40)

Original: HelloWorld
Encoded : [9, 32, 39, 39, 42, 24, 42, 45, 39, 31]
Decoded : HelloWorld
----------------------------------------
Original: ABCxyz
Encoded : [2, 3, 4, 51, 52, 53]
Decoded : ABCxyz
----------------------------------------
Original: ChatGPT
Encoded : [4, 35, 28, 47, 8, 17, 21]
Decoded : ChatGPT
----------------------------------------
Original: Hello, World!
Encoded : [9, 32, 39, 39, 42, 1, 1, 24, 42, 45, 39, 31, 1]
Decoded : Hello<unk><unk>World<unk>
----------------------------------------


Character-level tokenization solves the out-of-vocabulary problem but introduces new challenges:

1. Longer sequences: because each word becomes many tokens, models need to process much longer inputs.
2. Weaker semantic representation: individual characters carry very little meaning, so models must learn relationships across many steps.
3. Higher computational cost: longer sequences lead to more tokens per input, which increases training and inference time.

To find a better balance between vocabulary size and sequence length, we move to subword-level tokenization next.

### 1.3 - Subword-level tokenization

Sub-word methods such as `Byte-Pair Encoding (BPE)`, `WordPiece`, and `SentencePiece` **learn** common groups of characters and merge them into tokens. For example, the word **unbelievable** might turn into three tokens: **["un", "believ", "able"]**. This approach strikes a balance between word-level and character-level methods and fix their limitations.

The BPE algorithm builds a vocabulary iteratively using the following process:
1. Start with individual characters (each character is a token).
2. Count all adjacent pairs of tokens in a large text corpus.
3. Merge the most frequent pair into a new token.

Repeat steps 2 and 3 until you reach the desired vocabulary size (for example, 50,000 tokens).

In the next cell, you will experiment with BPE in practice to see how it compresses text into meaningful subword units. Instead of implementing the algorithm from scratch, you will use a pretrained tokenizer, which was already trained on a large text corpus to build its vocabulary, such as the data used to train `GPT-2`. This allows you to see how BPE works in practice with a real, learned vocabulary.

In [9]:
from transformers import AutoTokenizer

# Step 1: Load a pretrained GPT-2 tokenizer from Hugging Face.
# Refer to this to learn more: https://huggingface.co/docs/transformers/en/model_doc/gpt2

tokenizer = AutoTokenizer.from_pretrained("gpt2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
# Step 2: Use it to write encode and decode helper functions
def encode(text):
    return tokenizer.encode(text)

    pass


def decode(ids):
    return tokenizer.decode(ids)
    pass

In [11]:
# 3. Inspect the tokens to see how BPE breaks words apart.
sample = "Unbelievable tokenization powers! 🚀"

ids = encode(sample)
tokens = tokenizer.convert_ids_to_tokens(ids)

print("Text:   ", sample)
print("IDs:    ", ids)
print("Tokens: ", tokens)

Text:    Unbelievable tokenization powers! 🚀
IDs:     [3118, 6667, 11203, 540, 11241, 1634, 5635, 0, 12520, 248, 222]
Tokens:  ['Un', 'bel', 'iev', 'able', 'Ġtoken', 'ization', 'Ġpowers', '!', 'ĠðŁ', 'ļ', 'Ģ']


### 1.4 - TikToken

`tiktoken` is a fast, production-ready library for tokenization used by OpenAI models.
It is designed for efficiency and consistency with how OpenAI counts tokens in GPT models.

In this section, you will explore how different model families use different tokenizers. We will compare tokenizers used to train `GPT-2` and more powerful models such as `GPT-4`. By trying both, you will see how tokenization has evolved to handle more diverse text (including emojis, Unicode, and special characters) while remaining efficient.

In the next cell, you will use tiktoken to load these encodings and inspect how each one splits the same text. You may find reading this doc helpful: https://github.com/openai/tiktoken

In [12]:
import tiktoken

# Compare GPT-2 and GPT-4 tokenizers using tiktoken.

# Step 1: Load two tokenizers
enc_gpt2 = tiktoken.get_encoding("gpt2")
enc_gpt4 = tiktoken.encoding_for_model("gpt-4o")  # or "gpt-4"

# Step 2: Encode the same sentence with both and observe how they differ
sentence = "The 🌟 star-programmer implemented AGI overnight."

tokens_gpt2 = enc_gpt2.encode(sentence)
tokens_gpt4 = enc_gpt4.encode(sentence)

# Helper to show each token as text
def tokens_to_strings(encoding, token_ids):
    return [encoding.decode([tid]) for tid in token_ids]

print("Sentence:", sentence)
print("\nGPT-2:")
print("Token IDs:", tokens_gpt2)
print("Tokens   :", tokens_to_strings(enc_gpt2, tokens_gpt2))

print("\nGPT-4:")
print("Token IDs:", tokens_gpt4)
print("Tokens   :", tokens_to_strings(enc_gpt4, tokens_gpt4))


Sentence: The 🌟 star-programmer implemented AGI overnight.

GPT-2:
Token IDs: [464, 12520, 234, 253, 3491, 12, 23065, 647, 9177, 13077, 40, 13417, 13]
Tokens   : ['The', ' �', '�', '�', ' star', '-', 'program', 'mer', ' implemented', ' AG', 'I', ' overnight', '.']

GPT-4:
Token IDs: [976, 130321, 253, 8253, 81630, 1159, 20681, 19215, 40, 34454, 13]
Tokens   : ['The', ' �', '�', ' star', '-program', 'mer', ' implemented', ' AG', 'I', ' overnight', '.']


Try changing the input sentence and observe how different tokenizers behave.
Experiment with:
- Emojis, special characters, or punctuation
- Code snippets or structured text
- Non-English text (for example, Japanese, French, or Arabic)

If you are curious, you can also attempt to implement the BPE algorithm yourself using a small text corpus to see how token merges are learned in practice.

### 1.5 - Key Takeaways
- **Word-level**: simple and intuitive, but limited by large vocabularies and out-of-vocabulary issues
- **Character-level**: flexible and covers all text, but produces long sequences that are harder to model
- **Subword / BPE**: balances both worlds and is the default choice for most modern LLMs
- **TikToken**: a production-ready tokenizer used in OpenAI models, demonstrating how optimized subword vocabularies are applied in real systems

# 2. What is a Language Model?

At its core, a **language model (LM)** is just a *very large* mathematical function built from many neural-network layers.  
Given a sequence of tokens `[t₁, t₂, …, tₙ]`, it learns to output a probability for the next token `tₙ₊₁`.


Each layer performs basic mathematical operations such as matrix multiplication and attention. When hundreds of these layers are stacked together, the model learns complex patterns and statistical relationships in text. The final output is a vector of scores that represents how likely each possible token is to appear next. You can think of the entire model as one giant equation whose parameters were optimized during training to minimize prediction errors.

### 2.1 - A Single `Linear` Layer

Before jumping into Transformers, let's start with the simplest building block: a `Linear` layer.

A Linear layer computes `y = Wx + b`.

Where:  
  * `x` - input vector  
  * `W` - weight matrix (learned)  
  * `b` - bias vector (learned)

Although this operation looks simple, stacking many linear layers (along with nonlinear activation functions) allows neural networks to model highly complex relationships in data.

In the next cell, you will explore how a **Linear layer** works in practice by implementing one from scratch. You will define the weights and bias, then perform the matrix multiplication and addition manually to see what happens inside this layer. You may find the following links useful:
- https://docs.pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html
- https://docs.pytorch.org/docs/stable/generated/torch.randn.html
- https://docs.pytorch.org/docs/stable/generated/torch.matmul.html

In [13]:
import torch
import torch.nn as nn

# Define a MyLinear PyTorch module and perform y = Wx + b.

class MyLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(MyLinear, self).__init__()
        # Initialize weights and bias as learnable parameters
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias   = nn.Parameter(torch.randn(out_features))

    def forward(self, x):
        # Matrix multiplication followed by bias addition:  y = Wx + b
        return self.weight @ x + self.bias


lin = MyLinear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))

Input : tensor([ 1.0000, -1.0000,  0.5000])
Weights: Parameter containing:
tensor([[-0.8250,  1.3049,  0.1187],
        [-0.5890, -0.4155,  1.1964]], requires_grad=True)
Bias   : Parameter containing:
tensor([-1.0197,  0.7386], requires_grad=True)
Output : tensor([-3.0902,  1.1634], grad_fn=<AddBackward0>)


Next, you will use PyTorch's built-in nn.Linear module, which performs the same computation `(y = Wx + b)` but automatically handles parameter initialization, gradient tracking, and integration with the rest of a neural network. Comparing your manual implementation with this built-in version will help you understand what a linear layer does and how deep learning frameworks make these operations easier to use.

You may find this link useful:
- https://docs.pytorch.org/docs/stable/generated/torch.nn.Linear.html

In [14]:
import torch.nn as nn, torch

# Create a linear layer using pytorch's nn.Linear
lin = nn.Linear(3, 2)

x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))

Input : tensor([ 1.0000, -1.0000,  0.5000])
Weights: Parameter containing:
tensor([[-0.3170, -0.3254, -0.5291],
        [ 0.2270, -0.4149,  0.4038]], requires_grad=True)
Bias   : Parameter containing:
tensor([-0.5404, -0.1120], requires_grad=True)
Output : tensor([-0.7966,  0.7318], grad_fn=<ViewBackward0>)


### 2.2 - A `Transformer` Layer

Most LLMs are a **stack of identical Transformer blocks**. Each block fuses two main components:

| Step | What it does | Where it lives in code |
|------|--------------|------------------------|
| **Multi-Head Self-Attention** | Every token looks at every other token and decides *what matters*. | `block.attn` |
| **Feed-Forward Network (MLP)** | Re-mixes information token-by-token. | `block.mlp` |

In the next section, you will load `GPT-2` and inspect its first Transformer block to see these components in a real model. You will locate its layers, print their shapes and parameters, and understand how a block processes a batch of token embeddings.

In [15]:
import torch
from transformers import GPT2LMHeadModel

# Step 1: load the smallest GPT-2 model (124M parameters)
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Step 2: Inspect the first Transformer block
print(model.transformer.h[0])

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx=3072)
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)


In this section, you will run a minimal forward pass through one GPT-2 block to understand how tokens are transformed inside the model.

In [16]:
# Step 1: Create a small dummy input with a sequence of 8 random token IDs.
batch_size = 1
seq_len = 8
input_ids = torch.randint(0, model.config.vocab_size, (batch_size, seq_len))

# Step 2: Convert token IDs into embeddings
# GPT-2 uses two embedding layers:
#   - wte (word token embeddings)
#   - wpe (positional embeddings)
# Add them together to form the initial hidden representation of your input tokens.
position_ids = torch.arange(0, seq_len).unsqueeze(0)  # shape: (1, 8)
token_embeds = model.transformer.wte(input_ids)       # (1, 8, hidden_size)
pos_embeds = model.transformer.wpe(position_ids)      # (1, 8, hidden_size)
hidden_states = token_embeds + pos_embeds             # (1, 8, hidden_size)

# Step 3: Pass the embeddings through a single Transformer block
# This simulates one layer of computation in GPT-2.
hidden_states = model.transformer.h[0](hidden_states)[0]

# Step 4: Inspect the result
# The output shape should be (batch_size, sequence_length, hidden_size)
print(hidden_states.shape)

torch.Size([1, 8, 768])


### 2.3 - Inside GPT-2

GPT-2 is essentially a stack of identical Transformer blocks arranged in sequence.
Each block contains attention, feed-forward, and normalization layers that process token representations step by step.

In this section, you will print the modules inside the GPT-2 Transformer to see how these components are organized.
This will help you understand how the model scales from a single block to a full network of many layers working together.

In [17]:
# Print the name of all layers inside gpt.transformer.
# You may find this helpful: https://docs.pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.named_children

for name, layer in model.transformer.named_modules():
    print(name)


wte
wpe
drop
h
h.0
h.0.ln_1
h.0.attn
h.0.attn.c_attn
h.0.attn.c_proj
h.0.attn.attn_dropout
h.0.attn.resid_dropout
h.0.ln_2
h.0.mlp
h.0.mlp.c_fc
h.0.mlp.c_proj
h.0.mlp.act
h.0.mlp.dropout
h.1
h.1.ln_1
h.1.attn
h.1.attn.c_attn
h.1.attn.c_proj
h.1.attn.attn_dropout
h.1.attn.resid_dropout
h.1.ln_2
h.1.mlp
h.1.mlp.c_fc
h.1.mlp.c_proj
h.1.mlp.act
h.1.mlp.dropout
h.2
h.2.ln_1
h.2.attn
h.2.attn.c_attn
h.2.attn.c_proj
h.2.attn.attn_dropout
h.2.attn.resid_dropout
h.2.ln_2
h.2.mlp
h.2.mlp.c_fc
h.2.mlp.c_proj
h.2.mlp.act
h.2.mlp.dropout
h.3
h.3.ln_1
h.3.attn
h.3.attn.c_attn
h.3.attn.c_proj
h.3.attn.attn_dropout
h.3.attn.resid_dropout
h.3.ln_2
h.3.mlp
h.3.mlp.c_fc
h.3.mlp.c_proj
h.3.mlp.act
h.3.mlp.dropout
h.4
h.4.ln_1
h.4.attn
h.4.attn.c_attn
h.4.attn.c_proj
h.4.attn.attn_dropout
h.4.attn.resid_dropout
h.4.ln_2
h.4.mlp
h.4.mlp.c_fc
h.4.mlp.c_proj
h.4.mlp.act
h.4.mlp.dropout
h.5
h.5.ln_1
h.5.attn
h.5.attn.c_attn
h.5.attn.c_proj
h.5.attn.attn_dropout
h.5.attn.resid_dropout
h.5.ln_2
h.5.mlp
h.5.mlp.

As you can see, the Transformer holds various modules, arranged from a list of blocks (`h`). The following table summarizes these modules:

| Step | What it does | Why it matters |
|------|--------------|----------------|
| **Token → Embedding** | Converts IDs to vectors | Gives the model a numeric “handle” on words |
| **Positional Encoding** | Adds “where am I?” info | Order matters in language |
| **Multi-Head Self-Attention** | Each token asks “which other tokens should I look at?” | Lets the model relate words across a sentence |
| **Feed-Forward Network** | Two stacked Linear layers with a non-linearity | Mixes information and adds depth |
| **LayerNorm & Residual** | Stabilize training and help gradients flow | Keeps very deep networks trainable |


### 2.4 LLM's output

When you pass a sequence of tokens through a language model, it produces a tensor of logits with shape
`(batch_size, seq_len, vocab_size)`.
Each position in the sequence receives a vector of scores representing how likely every possible token is to appear next. By applying a softmax function on the last dimension, these logits can be converted into probabilities that sum to 1.

In the next cell, you will feed an 8-token dummy sequence into GPT-2, print the shape of its logits, and display the five most likely next tokens predicted for the final position in the sequence.


In [18]:
import torch, torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Step 1: Load GPT-2 model and its tokenizer
# Step 1: Load GPT-2 model and its tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [20]:
# Step 2: Tokenize input text
text = "Hello my name"

input_ids = tokenizer.encode(text, return_tensors="pt")

In [21]:
# Step 3: Pass the input IDs to the model
with torch.no_grad():
    outputs = model(input_ids)
logits = outputs.logits

In [22]:
# Step 4: Predict the next token
# We take the logits from the final position, apply softmax to get probabilities,
# and then extract the top 5 most likely next tokens.

# Logits for the last position in the sequence
last_logits = logits[0, -1, :]               # shape: (vocab_size,)

# Convert to probabilities
probs = F.softmax(last_logits, dim=-1)       # shape: (vocab_size,)

# Get top 5 token IDs and their probabilities
top_probs, top_ids = torch.topk(probs, k=5)

# Decode and print them
for token_id, prob in zip(top_ids, top_probs):
    token_str = tokenizer.decode([token_id.item()])
    print(f"Token: {repr(token_str):>10} | ID: {token_id.item():>5} | Prob: {prob.item():.4f}")

Token:      ' is' | ID:   318 | Prob: 0.7773
Token:        ',' | ID:    11 | Prob: 0.0373
Token:       "'s" | ID:   338 | Prob: 0.0332
Token:     ' was' | ID:   373 | Prob: 0.0127
Token:     ' and' | ID:   290 | Prob: 0.0076


### 2.5 - Key Takeaway

A language model is not a black box or something mysterious.
It is a large composition of simple, understandable layers such as linear layers, attention, and normalization, trained together to predict the next token in a sequence.

By learning this next-token prediction task at scale, the model gradually develops an internal understanding of language structure, meaning, and context, which allows it to generate coherent and relevant text.

# 3 - Text Generation (Decoding)
Once a language model has been trained to predict token probabilities, we can use it to generate text.
This process is called text generation or decoding.

At each step, the model outputs a probability distribution over possible next tokens.
A decoding algorithm then selects one token based on that distribution, appends it to the sequence, and repeats the process to build text word by word. Different decoding strategies control how the model chooses the next token and how creative or deterministic the output will be. For example:
- **Greedy** decoding: always pick the token with the highest probability. Simple and consistent, but often repetitive.
- **Top-k** or **Nucleus** (top-p) sampling: randomly sample from the top few likely tokens to add variety.
- Beam search: explores multiple candidate continuations and keeps the best overall sequence.

Note: `Temperature` adjusts randomness in sampling. Higher values make outputs more diverse, while lower values make them more focused and deterministic.

### 3.1 - Greedy decoding
In this section, you will use GPT-2 and Hugging Face's built-in generate method to produce text using the greedy decoding strategy.

In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


model_id = "gpt2"
device = "cuda" if torch.cuda.is_available() else "mps"

# Step 1. Load GPT-2 model and tokenizer.
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

# Step 2. Implement a text generation function using HuggingFace's generate method.
def generate(model, tokenizer, prompt, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
        )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [24]:
tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n GPT-2 | Greedy")
    print(generate(model, tokenizer, prompt, 80))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 GPT-2 | Greedy


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and

 GPT-2 | Greedy


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is 2+2?

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

2+2 is the number of times you can use a spell to cast a spell.

 GPT-2 | Greedy
Suggest a party theme.

The party theme is a simple, simple, and fun way to get your friends to join you.

The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple, simple, and fun way to get your friends to join you. The party theme is a simple, simple, and fun way to get your friends


Naively selecting the single most probable token at each step (known as greedy decoding) often leads to poor results in practice:
- Repetition loops: phrases like “The cat is is is…”
- Short-sighted choices: the most likely token right now might lead to incoherent text later

These issues are why more advanced decoding methods such as top-k and nucleus sampling are commonly used to make model outputs more diverse and natural.

### 3.2 - Top-k and top-p sampling
The generate function you implemented earlier can easily be extended to use different decoding strategies.

In this section, you will reimplement the same function but adapt it to support Top-k and Top-p (nucleus) sampling. These methods introduce controlled randomness, allowing the model to explore multiple plausible continuations instead of always choosing the single most likely next token.

In [31]:
def generate(model, tokenizer, prompt, strategy="greedy", max_new_tokens=128):
    # Tokenize and move to same device as the model
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Choose generation settings
    if strategy == "greedy":
        gen_kwargs = dict(do_sample=False)
    elif strategy == "top_k":
        gen_kwargs = dict(do_sample=True, top_k=50, top_p=1.0)
    elif strategy == "top_p":
        # nucleus sampling (top-p)
        gen_kwargs = dict(do_sample=True, top_p=0.9, top_k=0)
    else:
        raise ValueError(f"Unknown strategy: {strategy}")

    # Generate tokens
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            **gen_kwargs,
        )

    # Decode and return text
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [32]:
tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n GPT-2 | Top-p")
    print(generate(model, tokenizer, prompt, "top_p", 40))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 GPT-2 | Top-p


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time the virtues seemed to change, good spirits, health, and the qualities of conduct became as unhealthy as those of falsehood, but in the end what passes for order, material and spiritual, should still remain

 GPT-2 | Top-p


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is 2+2? How does 2+2 stack? Remember, 3+2 counts. It doesn't seem to affect the power, but 0 represents disadvantage. This would make 3+2 useless against 4-sided creatures

 GPT-2 | Top-p
Suggest a party theme.


Look for improvements in configuration scripts.

Use lodash.

Use lodash-sublisting.


Use mikkel.

Use mikkel.



### 3.3 - Try It Yourself

Now it’s time to experiment with text generation. Replace the sample prompts with your own prompts or adjust the decoding strategy.
You can experiment with:
- strategy: "greedy", "beam", "top_k", "top_p"
- temperature: values between 0.2 and 2.0
- k or p: thresholds that control sampling diversity

Try generating the same prompt with `greedy` and `top_p` (for example, 0.9). Notice how even small temperature changes can make the output more focused or more free-form.




# 4 - Completion vs. Instruction-tuned LLMs

So far, we have used `GPT-2` to generate text from a given input prompt. However, `GPT-2` is just a completion model. It simply continues the provided text without understanding it as a task or question. It is not designed to engage in dialogue or follow instructions.

In contrast, instruction-tuned LLMs (such as `Qwen-Chat`) undergo an additional post-training stage after base pre-training. This process fine-tunes the model to behave helpfully and safely when interacting with users. Because of this extra stage, instruction-tuned models can:

- Interpret prompts as requests rather than just text to continue
- Stay in conversation mode, answering questions and following steps
- Handle refusals and safety boundaries appropriately
- Maintain a consistent helpful persona, rather than drifting into storytelling

### 4.1 - `Qwen/Qwen3-0.6B` vs. `GPT2`

In the next cell, you will feed the same prompt to two different models:

- GPT-2 (completion-only): continues the text in the same writing style
- Qwen/Qwen3-0.6B (instruction-tuned): interprets the input as an instruction and responds helpfully

Comparing the two outputs will make the difference between completion and instruction-tuned behavior clear.



In [33]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Decide device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print("Using device:", device)

# ---- GPT-2 ----
gpt2_model_id = "gpt2"
gpt2_tokenizer = AutoTokenizer.from_pretrained(gpt2_model_id)
gpt2_model = AutoModelForCausalLM.from_pretrained(gpt2_model_id).to(device)

# ---- Qwen ----
# You can swap this out for whichever Qwen checkpoint your assignment specifies
qwen_model_id = "Qwen/Qwen2.5-0.5B"  # or e.g. "Qwen/Qwen2.5-1.5B"
qwen_tokenizer = AutoTokenizer.from_pretrained(qwen_model_id)
qwen_model = AutoModelForCausalLM.from_pretrained(qwen_model_id).to(device)

# Quick sanity prints (optional)
print("GPT-2 and Qwen models loaded.")

Using device: cuda


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

GPT-2 and Qwen models loaded.


We have now downloaded two small checkpoints: GPT-2 (124M parameters) and Qwen3-0.6B (600M parameters). If the previous cell took some time to run, that was mainly due to model download speed. The models will be cached locally, so future runs will be faster.

Next, we will generate text using our generate function with both models and the same prompt to directly compare how a completion-only model (GPT-2) behaves differently from an instruction-tuned model (Qwen).

In [34]:

tests=[("Once upon a time", "greedy"),("What is 2+2?", "top_k"),("Suggest a party theme.", "top_p")]
for prompt, strategy in tests:
    print(f"\nPrompt: {prompt!r} | Strategy: {strategy}")
    print(generate(gpt2_model, gpt2_tokenizer, prompt, strategy=strategy, max_new_tokens=40))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Prompt: 'Once upon a time' | Strategy: greedy


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger

Prompt: 'What is 2+2?' | Strategy: top_k


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is 2+2?

I see 1+2 as 1 plus 1+2. To the math: You can count 1+2 plus 1 + 1. Let's say, let's assume a set of 2

Prompt: 'Suggest a party theme.' | Strategy: top_p
Suggest a party theme. In my opinion, parties tend to show up when there are lots of other people present, rather than being tied to the event itself. If a party plans on doing a party sale that doesn't involve


# 5. (Optional) A Small Interactive LLM Playground
This section is optional. You do not need to implement it to complete the project. It is meant purely for exploration and will not significantly affect your core AI engineering skills.

If you are curious, you can build a simple interactive playground to experiment with text generation. You can:
- Create input widgets for the prompt, model selection, decoding strategy, and temperature
- Use Hugging Face's generate method to produce text based on the selected settings
- Display the model's response directly in the notebook output

You may find following links helpful:
- https://ipywidgets.readthedocs.io/en/latest/
- https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Steps to implement:
# 1. Load models and tokenizers (GPT-2 and Qwen).
# 2. Define a helper function to generate text with different decoding strategies.
# 3. Create interactive UI elements (prompt box, model selector, strategy selector, temperature slider).
# 4. Add a button to trigger text generation.
# 5. Define the button’s behavior.
# 6. Display the full UI for the playground.

"""
YOUR CODE HERE (~3-5 lines of code)
"""


## 🎉 Congratulations!

You've just learned, explored, and inspected a real **LLM**. In one project you:
* Learned how **tokenization** works in practice
* Used `tiktoken` library to load and experiment with most advanced tokenizers.
* Explored LLM architecture and inspected GPT2 blocks and layers
* Learned decoding strategies and used `top-p` to generate text from GPT2
* Loaded a powerful chat model, `Qwen3-0.6B` and generated text
* Built an LLM playground


👏 **Great job!** Take a moment to celebrate. You now have a working mental model of how LLMs work. The skills you used here power most LLMs you see everywhere.
